Gerardo Rivas Alvarez.

### Ejercicio 1

En clase vimos que la función `bits` permite ver la representación binaria de números de punto flotante (`Float64`) o de enteros (e.g., `Int64`). En particular, para números de punto flotante, el primer bit es el signo, los siguiente 11 son el *exponente* y el resto es la representación binaria (sin el 1 del principio), del número en cuestión.

La pregunta es ¿qué significa el exponente?

```julia
bits(0.1)
"0111111111110000000000000000000000000000000000000000000000000000"
```

In [20]:
bits(0.1) #jugando con bits.

"0011111110111001100110011001100110011001100110011001100110011010"

Citando textualmente a la Wikipedia "La representación de coma flotante (en inglés floating point, que significa «punto flotante») es una forma de notación científica usada en los microprocesadores con la cual se pueden representar números racionales extremadamente grandes y pequeños de una manera muy eficiente y compacta, y con la que se pueden realizar operaciones aritméticas." Así, la representación de un número de coma flotantes $m$ es $m=b \times 2^{r} $. Hay una convención internacional en la que se acuerda guardar en el primer bit la información del signo. En los siguientes 11 bits se guarda el exponente $r$ y los 52 restantes son información del coeficiente $b$. Probemos a ver qué tanto hace `bits()`

In [7]:
bits(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

O sea, que la representación del 1.0 es,el primer 0 es el signo positivo. Tenemos de exponente `01111111111` y `1` de coeficiente. Así que $1.0_{10} = 1_2 \times 10_2^{01111111111_2}$ cuando en realidad debería ser $1.0_{10}= 1_2 \times 10_2^0 $.

In [15]:
bits(2.0)

"0100000000000000000000000000000000000000000000000000000000000000"

Ahora, para la representación de $-2.0$ tenemos que el primer `1` nos da el signo negativo. El exponente es `10000000000`, así, $-2.0_{10} =-1_2 \times 10^{10000000000_2}_2  $. Cuando en realidad se debería escribir $-2.0_{10} = -1_2 \times 10^{1_2}_2$. Notemos que si se suma 1 al exponente del 1.0 tenemos $1111111111+1=10000000000$ que es el exponente de dos, como debe ser. Y si sumamos 1, deberíamos obtener el exponente del 4.0. El exponente del 4.0 es $10000000000+1 = 10000000001$.

In [16]:
bits(4.0)

"0100000000010000000000000000000000000000000000000000000000000000"

En efecto, el exponente en la representación del 4.0 es `10000000001`. cuando debería ser `10`, en base 2 naturalmente. Por lo tanto concluyo que la convención con la que representan en bits a los números con punto flotante tiene como cero del exponente no al cero, sino más bien al `01111111111`. 

### Ejercicio 2

- ¿Cuál es el número más chico que sumado (o restado) a 1.0 me da algo distinto de 1.0?
(Este número se bautiza como el "épsilon de la máquina")

- ¿Cuál es el número más chico que sumado (o restado) a 10.0 me da algo distinto de 10.0?


Para encontrar las épsilon de la máquina de 1.0 y 10.0 se implementó el siguiente código. Es un ciclo del 0 al 100 de uno en uno. En cada ciclo suma la potencia de dos correspondiente al negativo del contador del ciclo y evalúa cuál es la potencia con la que el resultado de la suma es 1.0 i.e. evalúa cuándo `1.0 + 2.0^(-i)==1.0`. En el caso en que esta proposición sea verdadera, la épsilon resulta ser $2.0^{-i+1}$ (si `i` es el paso en donde la suma no hace nada, entonces el paso anterior es el número más pequeño que al sumarlo da distinto). Análogamente con el 10.0. A diferencia del código anterior, donde usé base $e$ en lugar de base $2.0$, el resultado de este código coincide con las correspondientes `eps(1.0)` y `eps(10.0)`. Esto debido que a que el lenguaje natural de la computadora es el binario.

In [21]:
epsilon1 = 0
for i in 0:100
    if 1.0 + 2.0^(-i) == 1.0
        println("La épsilon de la máquina para 1.0 (uno) es ", 2.0^(-i+1))
        epsilon1= 2.0^(-i+1)
        break
    end
end



La épsilon de la máquina para 1.0 (uno) es 2.220446049250313e-16


In [22]:
epsilon10 = 234
for i in 0:100
    if 10.0 + 2.0^(-i) == 10.0
        println("La épsilon de la máquina para 10 (diez) es ", 2.0^(-i+1))
        epsilon10 = 2.0^(-i+1)
        break
    end
end

La épsilon de la máquina para 10 (diez) es 1.7763568394002505e-15


In [27]:
println("La razón entre la épsilon aquí calculada y la función eps(1.0) es ", epsilon1/eps(1.0), ". Así que son iguales.")

La razón entre la épsilon aquí calculada y la función eps(1.0) es 1.0. Así que son iguales.


In [30]:
println("La razón entre la épsilon aquí calculada y la función eps(10.0) es ", epsilon10/eps(10.0), ". Así que son iguales.")

La razón entre la épsilon aquí calculada y la función eps(10.0) es 1.0. Así que son iguales.


### Ejercicio 3

Define una functión llamada `derivada1` que implemente el cálculo de:

$$
\frac{\Delta f(x)}{\Delta x}(x_0) = \frac{f(x_0+h)-f(x_0)}{h}
$$

Obviamente esta función es una aproximación *finita* de la derivada. La pregunta es ¿qué tan buena es esta aproximación, pensando en términos de $h$?

- La primer sub-pregunta es ¿cómo se responde este tipo de pregunta?
- Uno tiene que tomar un caso concreto (para $f(x_0,h)$); sugiero empezar con un polinomio con coeficientes enteros, y después hacer el mismo tipo de pruebas con una función trascendental.
- ¿Hay algo *sorprendente* en el comportamiento de `derivada1` como función de $h$? ¿A qué lo puedes atribuir?
- ¿Puedes hacer que la función `derivada1` funcione para funciones más o menos arbitrarias, que el usuario define?
- ¿Puedes explicar de alguna manera los resultados que has obtenido?

A continuación se estudiará el comportamiento de `derivada1` con un par de ejemplos, se hará un ciclo que simulará el límite $h \to 0$ y se verá como cambia el error del cálculo conforme $h$ disminuye, tabulando los resultados.

In [24]:
#Se define la función derivada que calcula la derivada de una función con la definicón usual.
function derivada1(funcion,x_0,h)
    (funcion(x_0+h)-funcion(x_0))/h
end

derivada1 (generic function with 1 method)

In [39]:
# Análisis de error de derivada1 para sin(x).
g(x)=sin(x^3 - 1)
errorr(x,y)= abs(x-y)
println("Valor real \t h \t\t Derivada1 \t\t Error")
for i in 0:20
    println(3.0,"\t\t",10.0^(-i), "\t \t", derivada1(g,1.0,10.0^(-i)), "\t", errorr(3.0,derivada1(g,1.0,10.0^(-i))))
end
    
    

Valor real 	 h 		 Derivada1 		 Error
3.0		1.0	 	0.6569865987187891	2.343013401281211
3.0		0.1	 	3.2498890855922267	0.24988908559222667
3.0		0.01	 	3.0296363399299766	0.029636339929976607
3.0		0.001	 	3.002996486483527	0.0029964864835267946
3.0		0.0001	 	3.000299964985234	0.0002999649852339914
3.0		1.0e-5	 	3.00002999966094	2.9999660939861172e-5
3.0		1.0e-6	 	3.0000029997933577	2.999793357716385e-6
3.0		1.0e-7	 	3.0000003015117733	3.0151177332626844e-7
3.0		1.0e-8	 	3.0000000039720467	3.972046691558262e-9
3.0		1.0e-9	 	3.000000248221113	2.482211129972711e-7
3.0		1.0e-10	 	3.000000248221113	2.482211129972711e-7
3.0		1.0e-11	 	3.000000248221113	2.482211129972711e-7
3.0		1.0e-12	 	3.000266701747023	0.00026670174702303484
3.0		1.0e-13	 	2.9976021664879227	0.002397833512077341
3.0		1.0e-14	 	2.9976021664879227	0.002397833512077341
3.0		1.0e-15	 	3.330669073875469	0.3306690738754692
3.0		1.0e-16	 	0.0	3.0
3.0		1.0e-17	 	0.0	3.0
3.0		1.0e-18	 	0.0	3.0
3.0		1.0e-19	 	0.0	3.0
3.0		1.0e-20	 	0.0	

In [38]:
#Análisis de error de derivada1 para un polinomio.
f(x)=x + x^2 + x^3 + x^4 + x^5
errorr(x,y)= abs(x-y)
println("Valor real \t h \t\t Derivada1 \t\t Error")
for i in 0:20
    println(15.0,"\t\t",10.0^(-i), "\t \t", derivada1(f,1.0,10.0^(-i)), "\t", errorr(15.0,derivada1(f,1.0,10.0^(-i))))
end

Valor real 	 h 		 Derivada1 		 Error
15.0		1.0	 	57.0	42.0
15.0		0.1	 	17.156100000000016	2.1561000000000163
15.0		0.01	 	15.20150601000001	0.2015060100000099
15.0		0.001	 	15.020015005998033	0.020015005998033075
15.0		0.0001	 	15.002000150001749	0.0020001500017485796
15.0		1.0e-5	 	15.000200001580309	0.0002000015803087507
15.0		1.0e-6	 	15.00001999854561	1.99985456106333e-5
15.0		1.0e-7	 	15.000002004939006	2.0049390059284633e-6
15.0		1.0e-8	 	15.00000008647362	8.64736193761928e-8
15.0		1.0e-9	 	15.000001241105563	1.2411055632099988e-6
15.0		1.0e-10	 	15.000001241105565	1.2411055649863556e-6
15.0		1.0e-11	 	15.000001241105567	1.2411055667627124e-6
15.0		1.0e-12	 	15.001333508735115	0.0013335087351151742
15.0		1.0e-13	 	14.992451724538114	0.007548275461886078
15.0		1.0e-14	 	15.010215292932116	0.010215292932116427
15.0		1.0e-15	 	16.87538997430238	1.8753899743023794
15.0		1.0e-16	 	0.0	15.0
15.0		1.0e-17	 	0.0	15.0
15.0		1.0e-18	 	0.0	15.0
15.0		1.0e-19	 	0.0	15.0
15.0		1.0e-20	 	0.0	1

La teoría dice que el orden de magnitud de $h$ coincide con el orden de magnitud del error. Esto es cierto hasta $h$ del orde de e-8. Después el error crece, primero diferiendo en dos órdenes de magnitud, después en más de 15. Hasta que `derivada1` colapsa en 0.0. Esto pasa tanto para los polinomios, como para las funciones trascendentales como se ve en la dos tablas anteriores.  A continuación se estudia con detalle el comportamiento antes del colapso para el polinomio.

In [57]:
#Análisis de error de derivada1 para el mismo polinomio  de arriba, en un intervalo específico, más pequeño.

errorr(x,y)= abs(x-y)
println("Valor real \t h \t\t Derivada1 \t\t Error")
for i in 15:0.01:16
    println(15.0,"\t\t",10.0^(-i), "\t \t", derivada1(f,1.0,10.0^(-i)), "\t", errorr(15.0,derivada1(f,1.0,10.0^(-i))))
end

Valor real 	 h 		 Derivada1 		 Error
15.0		1.0e-15	 	16.87538997430238	1.8753899743023794
15.0		9.77237220955811e-16	 	13.633001291611984	1.3669987083880155
15.0		9.54992586021437e-16	 	13.950554685461006	1.0494453145389944
15.0		9.332543007969924e-16	 	14.27550484806168	0.7244951519383207
15.0		9.120108393559115e-16	 	14.608024072291443	0.3919759277085575
15.0		8.91250938133744e-16	 	14.94828866424445	0.051711335755550536
15.0		8.709635899560797e-16	 	15.296479036711172	0.2964790367111725
15.0		8.511380382023759e-16	 	15.652779804835996	0.6527798048359958
15.0		8.317637711026709e-16	 	16.017379884002377	1.017379884002377
15.0		8.128305161640995e-16	 	16.390472589998343	1.3904725899983426
15.0		7.943282347242821e-16	 	16.772255741515078	1.7722557415150781
15.0		7.762471166286927e-16	 	12.586149961024212	2.413850038975788
15.0		7.585775750291851e-16	 	12.879319054910756	2.1206809450892443
15.0		7.413102413009162e-16	 	13.179316934238209	1.8206830657617914
15.0		7.2443596007498905e-16	 	

Vemos que el error sigue creciendo hasta que la derivada1 colapsa en cero. Este es un error netamente computacional, puesto que el método númerico no predice este comportamiento. El error computacional se debe a la finitud con la que la computadora representa a los números reales.

Como conclusion de este ejercicio, este método se basa en la definición formal de la derivada. Tiene un error numérico intrínseco del orden de mágnitud de $h$ y otro error computacional debido a la represantación finita de los reales.

### Ejercicio 4

Este ejercicio es equivalente al anterior, pero considerando la función `derivada2` que implemente el cálculo de:

$$
\frac{\Delta f(x)}{\Delta x}(x_0) = \frac{f(x_0+h)-f(x_0-h)}{2h}
$$

¿Puedes explicar por qué la convergencia es mejor en este caso?


In [32]:
function derivada2(f, x_0, h)
    (f(x_0 + h)-f(x_0 - h))/2h
    
end

derivada2 (generic function with 1 method)

In [36]:
g(x)=sin(x^3-1)
errorr(x,y)= abs(x-y)
println("Valor real \t h \t\t Derivada1 \t\t Error")
for i in 0:20
    println(3.0,"\t\t",10.0^(-i), "\t \t", derivada2(g,1.0,10.0^(-i)), "\t", errorr(3.0,derivada2(g,1.0,10.0^(-i))))
end

Valor real 	 h 		 Derivada1 		 Error
3.0		1.0	 	0.7492287917633428	2.250771208236657
3.0		0.1	 	2.9634199130904193	0.0365800869095807
3.0		0.01	 	2.9996498402671934	0.0003501597328066097
3.0		0.001	 	2.999996499983777	3.5000162230680587e-6
3.0		0.0001	 	2.9999999649988918	3.50011082339563e-8
3.0		1.0e-5	 	2.9999999996473687	3.5263125752749147e-10
3.0		1.0e-6	 	2.999999999970745	2.9254820788082725e-11
3.0		1.0e-7	 	3.000000000086222	8.622214053843891e-11
3.0		1.0e-8	 	2.9999999928698164	7.1301835546933034e-9
3.0		1.0e-9	 	3.0000000816876593	8.168765930349764e-8
3.0		1.0e-10	 	3.000000248221113	2.482211129972711e-7
3.0		1.0e-11	 	3.000000248221113	2.482211129972711e-7
3.0		1.0e-12	 	3.0001001682933293	0.00010016829332926136
3.0		1.0e-13	 	2.9992675010248604	0.000732498975139606
3.0		1.0e-14	 	2.9976021664879227	0.002397833512077341
3.0		1.0e-15	 	3.1641356201816957	0.1641356201816957
3.0		1.0e-16	 	1.6653345369377348	1.3346654630622652
3.0		1.0e-17	 	0.0	3.0
3.0		1.0e-18	 	0.0	3.0
3.0		1

In [37]:
f(x)=x + x^2 + x^3 + x^4 + x^5
errorr(x,y)= abs(x-y)
println("Valor real \t h \t\t Derivada1 \t\t Error")
for i in 0:20
    println(15.0,"\t\t",10.0^(-i), "\t \t", derivada2(f,1.0,10.0^(-i)), "\t", errorr(15.0,derivada2(f,1.0,10.0^(-i))))
end

Valor real 	 h 		 Derivada1 		 Error
15.0		1.0	 	31.0	16.0
15.0		0.1	 	15.150100000000007	0.15010000000000723
15.0		0.01	 	15.001500010000024	0.0015000100000239769
15.0		0.001	 	15.000014999999145	1.4999999145004494e-5
15.0		0.0001	 	15.00000014999614	1.4999613995314576e-7
15.0		1.0e-5	 	15.000000001474943	1.4749428345339766e-9
15.0		1.0e-6	 	14.999999999432134	5.678657544194721e-10
15.0		1.0e-7	 	14.999999997655777	2.3442225938197225e-9
15.0		1.0e-8	 	14.999999953246856	4.6753143578825984e-8
15.0		1.0e-9	 	15.000000352927144	3.5292714350987353e-7
15.0		1.0e-10	 	15.000001241105565	1.2411055649863556e-6
15.0		1.0e-11	 	15.000001241105567	1.2411055667627124e-6
15.0		1.0e-12	 	15.000445330315415	0.00044533031541504897
15.0		1.0e-13	 	15.001333508735115	0.0013335087351151742
15.0		1.0e-14	 	15.010215292932116	0.010215292932116427
15.0		1.0e-15	 	15.987211554602252	0.9872115546022524
15.0		1.0e-16	 	8.881784197001252	6.118215802998748
15.0		1.0e-17	 	0.0	15.0
15.0		1.0e-18	 	0.0	15.0
15.0	

Si se desarrolla $f(x_0 + h)$ y $f(x_0 - h)$ en serie de Taylor y se hace la diferencia, se anula el error lineal en $h$ y el siguiente error apreciable va como $h^2$. Este resultado teórico se observa hasta $h\sim 10^{-6}$. Después, al igual que con `derivada1`, se observa un amuento en el error, debido a un error computacional.  

### Ejercicio 5

Manda el notebook con tus resultados haciendo un pull-request al repositorio del curso. Para hacer esto:
- Haz un `fork` del repositorio del curso a tu propia cuenta en GitHub
- Una vez que tengas resuelto el notebook, súbelo a **tu** repositorio (con `git push`), en la carpeta "tareas/usuario_github/nombre_notebook"
- Desde tu repositorio, haz un "Pull Request". El *pull request* sólo se aceptará si es legible en el sentido más amplio de la palabra y, obviamente, si es correcto. Esto significa que debe ser *entendible*, a partir de lo que ustedes escriben, incluso para gente que no está en el curso. (Como GitHub es público, piensen fríamente que esto puede ser leido por *muuucha* gente allá afuera.) Si el trabajo es en colaboración, háganlo saber en el propio notebook (cosa que se debe notar en el historial de lo que envíen). Finalmente, como código ético, incluyan cualquier fuente de inspiración, incluso cuando se trate del notebook de otro compañero/equipo del curso.

##Fuentes

- El [notebook](https://github.com/Ceboc/2016-1_TemasSelectosFisComp/blob/master/tareas/Ceboc/1-EjerciciosJulia.ipynb) de mi compañero Bertoni. 
- Métodos numéricos y computación. Ward Cheney, David Kincaid. CENGAGE learning.
- https://es.wikipedia.org/wiki/Coma_flotante